## Test NLP (fasttext)

### Lecture des données 

In [1]:
import sys, os, re, csv, codecs 
import numpy as np
import pandas as pd, configparser

# keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Input, LSTM, GlobalMaxPool1D, Bidirectional, Embedding, Dropout
from keras.models import Model


# sklearn
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

Using TensorFlow backend.


In [2]:
config = configparser.ConfigParser()
config.read('../config.cfg')
TRAIN_DATA_FILE=config['FILES']['TRAIN']
TEST_DATA_FILE=config['FILES']['TEST']

In [3]:
# Chargement des données sous forme de dataframes pandas
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

# Définition des différents labels disponibles sous forme de liste
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

y = train[list_classes].values
list_sentences_train = train["comment_text"].fillna("_na_").values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [4]:
embed_size = 300 # taille du vecteur
max_features = 20000 # nombre de mots uniques utilisés (i.e nombre de lignes dans le vecteur d'embedding)
maxlen = 100 # nombre maximum de mots à considérer dans un commentaire

#### Tokenisation

In [5]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

### Création de l'embedding

In [6]:
# On utilise un modèle fasttext préentrainé en francais (c'est mieux) pour avoir plus de vocabulaire
# https://fasttext.cc/docs/en/crawl-vectors.html
embedding_file = config['FILES']['FAST']

#### Lecture du fichier d'embedding

In [12]:
import io
word_index = tokenizer.word_index
words = word_index.keys()
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        # if tokens[0] in words:
        data[tokens[0]] = list(map(float, tokens[1:]))
    return data

# On récupère les vecteurs du fichier fasttext sous forme d'un dictionnaire 
embeddings_index = load_vectors(embedding_file)

In [13]:
len(embeddings_index)

999994

#### Création de la matrice d'embedding

In [14]:
# les vecteurs du modèles sont de dimension 50
vector_size = 300
vocabulary_size=min(len(word_index)+1,max_features)
embedding_matrix = np.zeros((vocabulary_size, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    try:
        embedding_vector = embeddings_index.get(word)
        embedding_matrix[i] = embedding_vector    
    except KeyError:
        vec = np.zeros(embed_size)
        embedding_matrix[i]=vec

### Modèle BLSTM

In [16]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 731s 5ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/2
143613/143613 [==============================] - 767s 5ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00


In [19]:
test_label = pd.read_csv(config['FILES']['LABEL'])
test_label_strip = test_label[test_label.toxic != -1]
yt = test_label_strip[list_classes].values

In [20]:
model.evaluate(X_te[test_label_strip.index], yt, batch_size=1024)

63978/63978 [==============================] - 52s 815us/step


[nan, 0.0]

In [21]:
y_pred_t = model.predict(X_te, batch_size=1024)

In [23]:
y_pred_t[0]

array([nan, nan, nan, nan, nan, nan], dtype=float32)

In [22]:
print(accuracy_score(yt, y_pred_t[test_label_strip.index].round()))

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [ ]:
# Classification pour chaque label de façon indépendante
for label in range(0,6):
    print('... Traitement du label {}'.format(list_classes[label]))
    # On calcule l'accuracy des prédictions
    acc = accuracy_score(yt[:,label], y_pred_t[test_label_strip.index, label].round())
    print('L\'accuracy du jeu test est {}'.format(acc))
    print('Matrice de confusion sur le jeu test:')
    print(confusion_matrix(yt[:,label], y_pred_t[test_label_strip.index, label].round()))
    print('Rapport : ')
    print(classification_report(yt[:,label], y_pred_t[test_label_strip.index, label].round()) )
          